歪打误撞，非自己决策的开仓——赚一笔就跑

原则：
1. 市场上的钱比你聪明的多，其追逐的热点就是正确

2. 只想我该如何进入热点，不要想着热点如何为我所处的领域所用

In [16]:
import tushare as ts

ts.set_token('409115656486e3f61481103ec280339511aee50ee9026f9b008985ea')
pro = ts.pro_api()

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


# 黄金和比特币趋势

## 比特币数据

In [31]:
ts.set_token("409115656486e3f61481103ec280339511aee50ee9026f9b008985ea")

pro = ts.pro_api()
df = pro.btc_pricevol(start_date='20180101', end_date='20180801')

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [32]:
df.head()

,date,price,volume
0,20180801,7579.056837,5.759918e+08
1,20180731,7915.267761,7.305237e+08
2,20180730,8128.549310,6.716885e+08
3,20180729,8193.888653,2.656748e+08
4,20180728,8168.527643,2.808106e+08


## 黄金数据

# 期货-股票联动
http://goodsfu.10jqka.com.cn

https://zhuanlan.zhihu.com/p/40802537

# 妖股反弹

# 跌停抄底

In [14]:
from pymongo import MongoClient

client = MongoClient()
db = client["stock"]
collection = db["jqk_concept"]

In [1]:
import tushare as ts
import pandas as pd

def get_daily_pro(trade_date):
    """
    应对旧接口get_today_all指令失效
    """
    trade_date = re.sub("-","",trade_date)
    ts.set_token('409115656486e3f61481103ec280339511aee50ee9026f9b008985ea')
    pro = ts.pro_api()
    df = pro.daily(trade_date=trade_date)
    df["code"] = df["ts_code"].str[:6]
    df["pct_change"] = df.apply(lambda x: round(x["close"] / x["pre_close"] * 100 - 100, 2), axis=1)
    df["low_pct_change"] = df.apply(lambda x: round(x["low"] / x["pre_close"] * 100 - 100, 2), axis=1)
    df["high_pct_change"] = df.apply(lambda x: round(x["high"] / x["pre_close"] * 100 - 100, 2), axis=1)
    df.drop(["change", "pct_chg", "ts_code"], axis=1, inplace=True)
    return df

In [2]:
import re

df = get_daily_pro("20200511")

c:\users\administrator\appdata\local\programs\python\python36\lib\site-packages\tushare\util\upass.py:28: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  return str(df.ix[0]['token'])


In [3]:
df.head()

,trade_date,open,high,low,close,pre_close,vol,amount,code,pct_change,low_pct_change,high_pct_change
0,20200511,3.17,3.20,3.05,3.09,3.16,83283.12,25846.541,000927,-2.22,-3.48,1.27
1,20200511,4.19,4.32,4.18,4.28,4.19,146603.26,62435.999,000928,2.15,-0.24,3.10
2,20200511,6.24,6.24,6.17,6.20,6.20,9958.01,6175.554,000929,0.00,-0.48,0.65
3,20200511,7.63,7.78,7.50,7.60,7.70,281786.65,214862.873,000930,-1.30,-2.60,1.04
4,20200511,9.06,9.12,8.95,8.98,9.05,44670.97,40268.924,000931,-0.77,-1.10,0.77


In [5]:
df.to_csv("D:/python_stock/20200511.csv")

In [6]:
from pymongo_functions import get_today_zt
from pymongo_class import *
from crawl_api import *
import pandas as pd
from general_functions import judge_date,JQK_concept_pct
from tushare_api import get_daily_pro
from keywords import concept_url,concept_list,daily_analysis_dir
from collections import defaultdict

class AnalysisZt():

    """
    分析每日涨停股对应概念
    """

    def __init__(self,min_zt_num=1,timeout=300):
        self.zt_collection = StockZtMongo()
        self.concept_collection = ConceptSummaryMongo()
        self.stock2month_collection = Stock2MonthMongo()
        self.concept2month_collection = JQKConcept2MonthMongo()
        self.min_zt_num = min_zt_num
        self.trade_date = judge_date()

    def update_stock2month(self,df):
        self.stock2month_collection.init_db()
        row_len = len(df)
        for row_num in range(row_len):
            content = df.iloc[row_num, :]
            self.stock2month_collection.update_daily(content["code"], content["pct_change"], content["low_pct_change"],
                              content["high_pct_change"], content["volume"], self.trade_date)

    def update_concept2month(self):
        self.concept2month_collection.init_db()


    def get_break_zt(self,df):
        """
        返回涨停回落个股数目
        """
        filter_df = df[(df["high"] / df["settlement"] >= 1.099) & (df["changepercent"] < 1.099)]
        return len(filter_df)

    def start_analysis(self):
        yesterday_max_zt = self.zt_collection.get_max_zt_days()
        df = get_daily_pro(self.trade_date)
        print("\n数据已获得")
        self.update_stock2month(df)
        zt_dict = get_today_zt(df)
        print("今日涨停回落股数%s" %(self.get_break_zt(df)))
        self.zt_collection.refresh()
        today_max_zt = self.zt_collection.get_max_zt_days()
        print("昨日涨停高度：%s，今日涨停高度：%s"%(yesterday_max_zt,today_max_zt))

ModuleNotFoundError: No module named 'pymongo_functions'